## Step 0: Loading Dependencies

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import re

## Step 1: Loading Data

In [ ]:
df = pd.read_excel(
    'UCS-Satellite-Database 5-1-2023.xlsx',
    engine='openpyxl'  
)

print("Shape:", df.shape)

Shape: (7560, 68)


## Step 2: Mapping Columns

In [ ]:
# mapping columns to a minimum quantity
col_map = {
    'Name of Satellite, Alternate Names' : 'sat_name',
    'Current Official Name of Satellite' : 'sat_name_official',
    'Country/Org of UN Registry' : 'country_registry',
    'Country of Operator/Owner' : 'country_operator',
    'Operator/Owner' : 'operator',
    'Users' : 'users',
    'Purpose' : 'purpose',
    'Detailed Purpose' : 'detailed_purpose',
    'Class of Orbit' : 'class_of_orbit',
    'Type of Orbit' : 'type_of_orbit',
    'Perigee(km)' : 'perigee_km',
    'Apogee(km)' : 'apogee_km',
    'Inclination(degrees)' : 'inclination_deg',
    'Date of Launch' : 'date_of_launch',
    'Launch Mass(kg)' : 'launch_mass_kg',
    'Expected Lifetime(yrs)' : 'expected_lifetime_yrs'
}
# appling mapping for columns that exist
existing_map = {k:v for k, v in col_map.items() if k in df.columns}
df = df.rename(columns = existing_map)

## Step 3: Keeping Relevant Columns

In [ ]:
# keeping relevant columns to avoid surprise columns
keep = [v for v in existing_map.values()]

# keeping raw name and purpose if they exist
for c in ["sat_name","operator","purpose","perigee_km","apogee_km","inclination_deg","date_of_launch"]:
    if c not in df.columns:
        df[c] = np.nan

In [6]:
df.dtypes

sat_name              object
sat_name_official     object
country_registry      object
country_operator      object
operator              object
                      ...   
Unnamed: 66           object
Unnamed: 67           object
perigee_km           float64
apogee_km            float64
inclination_deg      float64
Length: 71, dtype: object

## Step 4: Parsing Dates

In [ ]:
df['date_of_launch'] = pd.to_datetime(df['date_of_launch'], errors = 'coerce')

## Step 5: Fixing Data Types

In [ ]:
df['perigee_km'] = pd.to_numeric(df['perigee_km'], errors = 'coerce')

In [ ]:
df['apogee_km'] = pd.to_numeric(df['apogee_km'], errors = 'coerce')

In [ ]:
df['inclination_deg'] = pd.to_numeric(df['inclination_deg'], errors = 'coerce')

In [ ]:
df['launch_mass_kg'] = pd.to_numeric(df.get('launch_mass_kg'), errors = 'coerce')

## Step 6: Feature Engineering

In [ ]:
# deriving perigee and apogee
df['mean_alt_km'] = df[['perigee_km','apogee_km']].mean(axis = 1)

In [ ]:
# deriving the year of launch
df['launch_year'] = df['date_of_launch'].dt.year

In [ ]:
# pinning analysis date
ANALYSIS_DATE = datetime(2025, 9, 6) 

In [ ]:
# creating age feature
df['age_years'] = (ANALYSIS_DATE - df['date_of_launch']).dt.days / 365.25

In [ ]:
# deriving orbit shells
def classify_shell(alt, text):
    if pd.notna(alt):
        if alt < 2000: return 'LEO'
        if alt < 35786: return 'MEO'
        return 'GEO'
    if isinstance(text, str):
        s=text.lower()
        if 'leo' in s: return 'LEO'
        if 'meo' in s: return 'MEO'
        if 'geo' in s: return 'GEO'
    return 'UNKNOWN'
df['orbit_shell'] = df.apply(lambda r: classify_shell(r['mean_alt_km'], r.get('class_of_orbit','')), axis = 1)

In [ ]:
# bucketing inclination into bands
def incl_band(x):
    if pd.isna(x): return 'UNKNOWN'
    x = float(x)
    if x < 30: return 'Equatorial (0-30)'
    if x < 60: return 'Low-mid (30-60)'
    return 'Polar/high (60-90)'
df['inclination_band'] = df['inclination_deg'].apply(incl_band)

In [ ]:
# canonicalizing purpose
purpose_map = {'comm':'Communication','communicat':'Communication','communication':'Communication',
               'earth': 'EO','eo':'EO','remote':'EO','science':'Scientific','scientific':'Scientific',
               'milit':'Military','nav':'Navigation','tech':'Technology/Demo'}
def canon_purpose(p):
    if pd.isna(p): return 'Unknown'
    s = str(p).lower()
    for k,v in purpose_map.items():
        if k in s: return v
    return s.title()
df['purpose_canonical'] = df['purpose'].apply(canon_purpose)

In [ ]:
# bucketing altitude into 1,000-km bands
df['alt_band_1000km'] = (df['mean_alt_km']//1000).fillna(-1).astype(int)*1000

In [ ]:
# flagging aged hardware (10 and 15 year thresholds)
df['old_10y'] = df['age_years'] > 10
df['old_15y'] = df['age_years'] > 15

## Step 7: Exporting Cleaned Data

In [ ]:
df.to_csv('ucs_satellites_cleaned.csv', index = False)